# UnifyML Quickstart

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/holl-/UnifyML/blob/main/docs/Introduction.ipynb) 
&nbsp; • &nbsp; [🌐 **UnifyML**](https://github.com/holl-/UnifyML)
&nbsp; • &nbsp; [📖 **Documentation**](https://holl-.github.io/UnifyML/unifyml/)
&nbsp; • &nbsp; [🔗 **API**](https://holl-.github.io/UnifyML/unifyml)
&nbsp; • &nbsp; [**▶ Videos**]()
&nbsp; • &nbsp; [<img src="images/colab_logo_small.png" height=4>](https://colab.research.google.com/github/holl-/UnifyML/blob/main/docs/Examples.ipynb) [**Examples**](https://holl-.github.io/UnifyML/Examples.html)



## Installation

Install UnifyML with [pip](https://pypi.org/project/pip/) on [Python 3.6](https://www.python.org/downloads/) and later:

In [2]:
# !pip install unifyml

Install [PyTorch](https://pytorch.org/), [TensorFlow](https://www.tensorflow.org/install) or [Jax](https://github.com/google/jax#installation) to enable machine learning capabilities and GPU execution.
See the [detailed installation instructions](https://holl-.github.io/UnifyML/Installation_Instructions.html).

In [3]:
from unifyml import math

## Usage without UnifyML's Tensors

You can call many functions on native tensors directly.
UnifyML will dispatch the call to the corresponding library and return the result as another native tensor.

In [4]:
math.sin(1.)

0.841471

In [5]:
from jax import numpy as jnp
math.sin(jnp.asarray([1.]))

DeviceArray([0.841471], dtype=float32)

In [6]:
import torch
math.sin(torch.tensor([1.]))

tensor([0.8415], device='cuda:0')

In [7]:
import tensorflow as tf
math.sin(tf.constant([1.]))

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.84147096], dtype=float32)>

In [8]:
import numpy as np
math.sin(np.asarray([1.]))

array([0.841471], dtype=float32)

## UnifyML's `Tensor`

For more advanced operations, we recommend using [UnifyML's tensors](Tensors.html).
While UnifyML includes a [unified low-level API](https://holl-.github.io/UnifyML/unifyml/backend/#unifyml.backend.Backend) that behaves much like NumPy, using it correctly (so that the code is actually compatible with all libraries) is difficult.
Instead, UnifyML provides a higher-level API consisting of the [`Tensor` class](https://holl-.github.io/UnifyML/unifyml/math/#unifyml.math.Tensor), the [`math`](https://holl-.github.io/UnifyML/unifyml/math) functions and other odds and ends, that makes writing unified code easy.
Tensors can be created by wrapping an existing backend-specific tensor or array:


In [9]:
torch_tensor = torch.tensor([1, 2, 3])
math.tensor(torch_tensor)

(1, 2, 3) int64

In [10]:
math.wrap(torch_tensor)

(1, 2, 3) int64

The difference between `tensor` and `wrap` is that `wrap` keeps the original data you pass in while `tensor` will convert the data to the default backend which can be set using [`math.use()`](https://holl-.github.io/UnifyML/unifyml/math/#unifyml.math.use).

In [11]:
math.use('jax')
math.wrap(torch_tensor).default_backend

torch

In [12]:
math.tensor(torch_tensor).default_backend

jax

The last `tensor` call converted the PyTorch tensor to a Jax `DeviceArray` using a no-copy routine from [`dlpack`](https://github.com/dmlc/dlpack) under the hood.

## Dimension Types

For tensors with more than one dimensions, you have to specify a name and type for each.
Possible types are *batch* for parallelizing code, *channel* for listing features (color channels or x/y/z components) and *spatial* for equally-spaced sample points (width/height of an image, 1D time series, etc.).
For an exhaustive list, see [here](Shapes.html)

In [13]:
from unifyml.math import batch, spatial, channel
torch_tensor = torch.tensor([[1, 2, 3], [4, 5, 6]])
math.wrap(torch_tensor, batch('dim1'), channel('dim2'))

(1, 2, 3); (4, 5, 6) (dim1ᵇ=2, dim2ᶜ=3) int64

When creating a new tensor from scratch, we also need to specify the size along each dimension:

In [14]:
math.random_uniform(batch(dim1=2), channel(dim2=3))

(0.072, 0.020, 0.077); (0.879, 0.165, 0.102) (dim1ᵇ=2, dim2ᶜ=3)

When passing tensors to a neural network, the tensors are transposed to match the preferred dimension order (`BHWC` for TensorFlow/Jax, `BCHW` for PyTorch).
For example, we can pass any number of batch and channel dimensions to an MLP.

In [15]:
from unifyml import nn
mlp = nn.mlp(in_channels=6, out_channels=3, layers=[64, 64])
data = math.random_normal(batch(b1=4, b2=10), channel(c1=2, c2=3))
math.native_call(mlp, data)

(b1ᵇ=4, b2ᵇ=10, vectorᶜ=3) -1.04e-04 ± 3.0e-01 (-1e+00...9e-01)

The network here is a standard fully-connected network module with two hidden layers of 64 neurons each.
The native tensor that is passed to the network has shape (40, 6) as all batch dimensions are compressed into the first and all channel dimensions into the last dimension.

For a network acting on spatial data, we would add *spatial* dimensions.

In [17]:
net = nn.u_net(in_channels=6, out_channels=3, in_spatial=2)
data = math.random_normal(batch(b1=4, b2=10), channel(c1=2, c2=3), spatial(x=28, y=28))
math.native_call(mlp, data)

(b1ᵇ=4, b2ᵇ=10, xˢ=28, yˢ=28, vectorᶜ=3) -0.004 ± 0.322 (-2e+00...2e+00)

In this example, we ran a 2D [U-Net](https://en.wikipedia.org/wiki/U-Net#:~:text=U%2DNet%20is%20a%20convolutional,of%20the%20University%20of%20Freiburg.).
For a 1D or 3D variant, we would pass `in_spatial=1` or `3`, respectively, and add the corresponding number of spatial dimensions to `data`.

## Further Reading

While the dimensionality of neural networks must be specified during network creation, this is not the case for math functions.
These [automatically adapt to the number of spatial dimensions of the data that is passed in](N_Dimensional.html).

If you want to get deeper into UnifyML, check out the following notebooks:

* [Shapes](Shapes.html)
* [More on tensors](Tensors.html)
* [Data types](Data_Types.html)
* [Writing *n*-dimensional code](N_Dimensional.html)

&nbsp; • &nbsp; [🌐 **UnifyML**](https://github.com/holl-/UnifyML)
&nbsp; • &nbsp; [📖 **Documentation**](https://holl-.github.io/UnifyML/unifyml/)
&nbsp; • &nbsp; [🔗 **API**](https://holl-.github.io/UnifyML/unifyml)
&nbsp; • &nbsp; [**▶ Videos**]()
&nbsp; • &nbsp; [<img src="images/colab_logo_small.png" height=4>](https://colab.research.google.com/github/holl-/UnifyML/blob/main/docs/Examples.ipynb) [**Examples**](https://holl-.github.io/UnifyML/Examples.html)